In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dataPrep as dp
from scipy.stats import mode
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score

ModuleNotFoundError: No module named 'dataPrep'

In [ ]:
df = pd.read_csv('Prostate_Cancer.csv')
df.head()

In [ ]:
encoder=LabelEncoder()
df['diagnosis_result'] = encoder.fit_transform(df['diagnosis_result'])

In [ ]:
dp.outliers_imputation(df,'perimeter','mean')
dp.outliers_imputation(df,'smoothness','mean')
dp.outliers_imputation(df,'compactness','mean')
dp.outliers_imputation(df,'symmetry','mean')
dp.outliers_imputation(df,'fractal_dimension','mean')

In [ ]:
Y_data = df['diagnosis_result'].values
X_data = df.drop(['diagnosis_result'],axis=1).values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_data,Y_data,test_size=0.2,random_state=0)


In [ ]:
pd.DataFrame(x_test)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_data = scaler.fit_transform(X_data)

In [ ]:
class KNNClassifier:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        y_pred = [self.getEculideanDistance(x) for x in X_test]
        return np.array(y_pred)

    def getEculideanDistance(self, x):
        # get the distances between x and x_train 
        distances = [np.linalg.norm(x - x_train) for x_train in self.X_train]
        
        # Get the indices of k-nearest points of the training data 
        k_indices = np.argsort(distances)[:self.k]
        
        # Get the labels of the k-nearest training data points
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        
        # Return the most common class label
        commonClasses = np.bincount(k_nearest_labels).argmax()
        return commonClasses
    def CM(self,y_test,y_pred):
        # get the confusion matrix and get the true negative,true positive ,false positive and false negative

        cm=[]
        TP=[]
        TN=[]
        FP=[]
        FN=[]

        for i in range(len(y_test)):
            if(y_test[i]==1 and y_pred[i]==1):
                TN.append(1)
            if(y_test[i]==0 and y_pred[i]==1):
                FP.append(1)
            if(y_test[i]==1 and y_pred[i]==0):
                FN.append(1)
            if(y_test[i]==0 and y_pred[i]==0):
                TP.append(1)

        sum_TP=sum(TP)
        sum_TN=sum(TN)
        sum_FP=sum(FP)
        sum_FN=sum(FN)

        cm.append(sum_TP)
        cm.append(sum_FP)
        cm.append(sum_FN)
        cm.append(sum_TN)
        
        print("precision")
        print((sum_TP)/(sum_FP+sum_TP))
        print("acuracy")
        print((sum_TP+sum_TN)/(sum_TP+sum_TN+sum_FN+sum_FP))
        print("Sensitivity")
        print((sum_TP)/(sum_FN+sum_TP))
        print("specficity")
        print((sum_TN)/(sum_FP+sum_TN))
        print("confusion matrix")
        print(cm)

knn = KNNClassifier(k=12)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
accu=accuracy_score(y_test,y_pred)
knn.CM(y_test,y_pred)